In [ ]:
import torch 
torch.cuda.get_device_name(device=None)

---
---
# BOSCH AI HACKATHON 2021
## Team - KanthabAI

Members:
  - Sanjana
  - Lakshya
  - Abeesh
  - Sachin
  - Shubham
  
~ Code, data and scripts maintained by Lakshya Dev at https://github.com/lakshyads/bosch-ai-hackathon-2021-kanthabai

---
---

## 1. PREP THE ENVIRONMENT AND DATA
---

### Setup git LFS for downloading dataset with repo

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs

### Cloning the model and script files from Github repo

In [ ]:
!git clone https://github.com/lakshyads/bosch-ai-hackathon-2021-kanthabai.git yolov5

In [ ]:
!git pull

### Ready the dataset included with above repo (extract & split)



In [ ]:
%cd /content/yolov5/

# Unzip dataset
!unzip -q ./data/final-data-full.zip -d /content/yolov5/data/dataset/

#split dataset
!python /content/yolov5/data-utils/split_train_val_test.py --images /content/yolov5/data/dataset --labels /content/yolov5/data/dataset --out /content/yolov5/data/dataset --move y

### Installing the dependencies

In [ ]:
!pip install -r requirements.txt

%cd /content/yolov5/

### Option to connect and use Google drive


In [ ]:
useGDrive = False

if (useGDrive == True):
  %cd ..
  !pip install PyDrive

  import os
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials

  # Connect GDrive
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  # Download zipped data to Colab from Google Drive
  download = drive.CreateFile({'id': '1S23A8_uuiE2zp50qJDSp_xcC294kaFW7'})
  download.GetContentFile('train-validate-test-split-data.zip')

  %cd /content/yolov5
else:
  print("Google Drive will not be used. set useGDrive = True to use G Drive")

### Give full permissions to everthing inside /content/yolov5

In [ ]:
!chmod -R 755 /content/yolov5


### Download all versions of YOLO-V5 model

In [ ]:
!/content/yolov5/weights/download_weights.sh

---
## 2. TRAINING THE YOLO-V5 MODEL
---

### Run training

In [ ]:
!python train.py --img 640 --batch 16 --epochs 1 --data final-data.yaml --weights yolov5s6.pt --nosave --cache 

### Commit run folder updates to git

In [ ]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d%m%Y-T-%H%M%S")
model_name = 'yolov5s6'
epocs = '150'
batch_size = '16'
initial_weights = 'pre_Init_Weights'
# initial_weights = 'custom_Init_Weights'


### Zip and upload complete runs folder to GDrive as precaution

In [ ]:
# Backup run folder to Google drive
backup_to_drive = False

if(backup_to_drive is True):
  print('Backing up to google drive')
  import shutil
  import os

  bak_archive_name = f'run-{model_name}-{initial_weights}-epocs_{epocs}-batch_{batch_size}-{dt_string}'
  bak_archive_path = r'/content/yolov5/backups-for-gdrive/'
  bak_archive = os.path.join(bak_archive_path, bak_archive_name)
  dir_to_archive = '/content/yolov5/runs'
  try:
    print('\n Trying to upload run folder to gdrive ...')
    if not os.path.exists(bak_archive_path):
      os.makedirs(bak_archive_path)

    shutil.make_archive(base_name=bak_archive, root_dir=dir_to_archive, format= 'zip', base_dir=dir_to_archive)

    file = drive.CreateFile({'title': f'{bak_archive_name}.zip', 'parents': [{'id': '1Fo9h4-e-AvGSeYX0Q_086MnP7uquJMyN'}]})
    file.SetContentFile(f'{bak_archive}.zip')
    file.Upload()  
    print(f'\n Run data uploaded to google drive. Uploaded file name = {bak_archive}')
  except Exception as e:
    print(f'\n Failed to upload Run data to GDrive: {bak_archive} to Google Drive. \nException details: {e}')

# VISUALIZING THE TRAINING METRICS

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/train

## PLOTTING TRAINING RESULTS

In [ ]:
from utils.plots import plot_results

plot_results(save_dir='runs/train/exp3')

# TESTING

In [ ]:
!python test.py --weights /content/yolov5/runs/train/exp3/weights/best.pt --data my_data.yaml --img 640

## Extract new images from zipped upload


In [ ]:
%cd ..
!unzip -q /content/extracted_images.zip 
%cd yolov5

# DETECTING ON NEW IMAGES

In [ ]:
!python detect.py --source /content/extracted_images --weights /content/last.pt --img 640 --save-txt --save-conf

#RETRAINING FROM THE LAST CHECKPOINT

In [ ]:
!python train.py --weights /content/yolov5/runs/train/exp2/weights/last.pt --epochs 100 --img 416

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re
path='/content/yolov5/runs/detect/exp3/labels/11_image.txt'
myfile=open(path,'r')
lines=myfile.readlines()
pattern= "class_name"

cordinate = []
for line in lines:
  for x in line.splitlines():
    cordinate.append([x[1:]])
#   if re.search(pattern,line):
#     Cord_Raw=line
# Cord=Cord_Raw.split("(")[1].split(")")[0].split("  ")

In [ ]:
cordinate

In [ ]:
cordinate[0]

In [ ]:

import cv2
img = cv2.imread("/content/drive/MyDrive/images/train/11_image.jpg")
print(img.shape)
for i in cordinate:
  count= 0
  for val in i:
    value = val.strip().split(" ")
    x_min = int(float(value[0])*img.shape[0])
    x_max = x_min + int(float(value[2])*img.shape[0])
    y_min = int(float(value[1])*img.shape[1])
    y_max = y_min + int(float(value[3])*img.shape[1])
    crop_img = img[y_min:y_max, x_min:x_max]
    cv2.imwrite(f"Object_{count}.jpg",crop_img)
    count = count + 1





In [ ]:
import cv2
img = cv2.imread("/content/drive/MyDrive/images/train/11_image.jpg")
crop_img = img[y_min:y_max, x_min:x_max]
cv2.imwrite("Object.jpg",crop_img)